<a href="https://colab.research.google.com/github/sruby8/uplift.ai/blob/master/April29_PitchingSkeleton_uplift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Multi-View 3D Skeleton Animation Script with Throwing Arm Auto-Detection

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter, PillowWriter
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
import re

# === SETTINGS ===
file_path = "./uplift_data_export_20250427183322.csv"
output_format = "mp4"
slow_motion_fps = 5

views_choice = 'all'

views_available = {
    "behind_pitcher": 315,
    "first_base_side": 135,
    "third_base_side": 225,
    "home_plate_view": 45,
}

if views_choice == 'all':
    views_to_render = views_available
else:
    views_to_render = {view: views_available[view] for view in views_choice}

# === Load CSV ===
df = pd.read_csv(file_path)

# Determine handedness and flip X if right-handed
pitcher_handedness = df['handedness'].dropna().iloc[0].lower() if 'handedness' in df.columns else 'left'
flip_x = pitcher_handedness == 'left'

# Auto-detect dominant arm based on wrist velocity
right_cols = ['right_wrist_jc_3d_x', 'right_wrist_jc_3d_y', 'right_wrist_jc_3d_z']
left_cols = ['left_wrist_jc_3d_x', 'left_wrist_jc_3d_y', 'left_wrist_jc_3d_z']
right_vel = np.linalg.norm(np.diff(df[right_cols].values, axis=0, prepend=df[right_cols].values[0:1]), axis=1)
left_vel = np.linalg.norm(np.diff(df[left_cols].values, axis=0, prepend=df[left_cols].values[0:1]), axis=1)
thrw_joint = 'right_wrist_jc_3d' if right_vel.sum() > left_vel.sum() else 'left_wrist_jc_3d'

# Map and remap joints
joint_columns = [col for col in df.columns if any(suffix in col.lower() for suffix in ['_x', '_y', '_z'])]
joint_groups = defaultdict(list)
for col in joint_columns:
    base = re.sub(r'_(x|y|z)$', '', col.lower())
    joint_groups[base].append(col)
full_triplet_joints = {joint: coords for joint, coords in joint_groups.items() if len(coords) == 3}

core_joints = [
    'left_shoulder_jc_3d', 'left_elbow_jc_3d', 'left_wrist_jc_3d',
    'right_shoulder_jc_3d', 'right_elbow_jc_3d', 'right_wrist_jc_3d',
    'left_hip_jc_3d', 'left_knee_jc_3d', 'left_ankle_jc_3d',
    'right_hip_jc_3d', 'right_knee_jc_3d', 'right_ankle_jc_3d'
]
core_joints = [j for j in core_joints if j in full_triplet_joints]

def get_xyz_remapped(joint):
    x = df[f"{joint}_x"].values
    y = df[f"{joint}_y"].values
    z = df[f"{joint}_z"].values
    x = -x if flip_x else x
    return np.stack([x, z, y], axis=1)

left_shoulder = get_xyz_remapped("left_shoulder_jc_3d")
right_shoulder = get_xyz_remapped("right_shoulder_jc_3d")
left_hip = get_xyz_remapped("left_hip_jc_3d")
right_hip = get_xyz_remapped("right_hip_jc_3d")
spine_top = (left_shoulder + right_shoulder) / 2
spine_base = (left_hip + right_hip) / 2

core_joints += ['spine_top_virtual', 'spine_base_virtual']

skeleton_frames = []
for joint in core_joints:
    if "virtual" in joint:
        skeleton_frames.append(spine_top if "top" in joint else spine_base)
    else:
        skeleton_frames.append(get_xyz_remapped(joint))
skeleton_frames = np.stack(skeleton_frames, axis=1)

velocity_frames = np.linalg.norm(np.diff(skeleton_frames, axis=0, prepend=skeleton_frames[0:1]), axis=2)
velocity_normed = (velocity_frames - velocity_frames.min()) / (velocity_frames.max() - velocity_frames.min())

bones = [
    ("left_shoulder_jc_3d", "left_elbow_jc_3d"),
    ("left_elbow_jc_3d", "left_wrist_jc_3d"),
    ("right_shoulder_jc_3d", "right_elbow_jc_3d"),
    ("right_elbow_jc_3d", "right_wrist_jc_3d"),
    ("left_hip_jc_3d", "left_knee_jc_3d"),
    ("left_knee_jc_3d", "left_ankle_jc_3d"),
    ("right_hip_jc_3d", "right_knee_jc_3d"),
    ("right_knee_jc_3d", "right_ankle_jc_3d"),
    ("left_shoulder_jc_3d", "right_shoulder_jc_3d"),
    ("left_hip_jc_3d", "right_hip_jc_3d"),
    ("spine_base_virtual", "spine_top_virtual"),
    ("spine_base_virtual", "left_hip_jc_3d"),
    ("spine_base_virtual", "right_hip_jc_3d"),
    ("spine_top_virtual", "left_shoulder_jc_3d"),
    ("spine_top_virtual", "right_shoulder_jc_3d"),
]

frame_indices = np.linspace(0, skeleton_frames.shape[0] - 1, 50, dtype=int)
ball_release_frame = df[df['ball_release_frame'] == 0].index.min() if 'ball_release_frame' in df.columns else None

for view_name, azim_angle in views_to_render.items():
    adjusted_azim = 360 - azim_angle if flip_x else azim_angle

    plt.style.use("dark_background")
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    def update(frame_idx):
        frame = frame_indices[frame_idx]
        ax.clear()
        ax.set_xlim(2, -2)
        ax.set_ylim(-2, 2)
        ax.set_zlim(-0.5, 2.5)
        ax.view_init(elev=15, azim=adjusted_azim)
        ax.set_facecolor('black')
        ax.set_axis_off()

        for j1, j2 in bones:
            a, b = core_joints.index(j1), core_joints.index(j2)
            x = [skeleton_frames[frame, a, 0], skeleton_frames[frame, b, 0]]
            y = [skeleton_frames[frame, a, 1], skeleton_frames[frame, b, 1]]
            z = [skeleton_frames[frame, a, 2], skeleton_frames[frame, b, 2]]
            ax.plot(x, y, z, color='white', linewidth=3)

        for j in range(len(core_joints)):
            x, y, z = skeleton_frames[frame, j]
            v = velocity_normed[frame, j]
            ax.scatter(x, y, z, color=plt.cm.plasma(v), s=50, edgecolors='black', linewidths=0.5)

        wrist_idx = core_joints.index(thrw_joint)
        trail_length = 10
        past_idxs = frame_indices[max(0, frame_idx - trail_length):frame_idx + 1]
        alphas = np.linspace(0.1, 1.0, len(past_idxs))
        for i in range(1, len(past_idxs)):
            f1, f2 = past_idxs[i - 1], past_idxs[i]
            p1, p2 = skeleton_frames[f1, wrist_idx], skeleton_frames[f2, wrist_idx]
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]],
                    color='red', linewidth=2, alpha=alphas[i])

        if ball_release_frame is not None and frame == ball_release_frame:
            x, y, z = skeleton_frames[frame, wrist_idx]
            ax.scatter(x, y, z, s=100, c='lime', marker='*')
            ax.text(x, y, z + 0.2, 'Ball Release', color='lime', fontsize=10)

    output_filename = f"pitching_skeleton_{pitcher_handedness}_{view_name}.{output_format}"
    ani = FuncAnimation(fig, update, frames=len(frame_indices), interval=1000 // slow_motion_fps)

    if output_format == "gif":
        ani.save(output_filename, writer=PillowWriter(fps=slow_motion_fps))
    else:
        ani.save(output_filename, writer=FFMpegWriter(fps=slow_motion_fps))

    plt.close(fig)

print("✅ Throwing arm auto-detected and views rendered with correct trail and handedness!")


✅ Throwing arm auto-detected and views rendered with correct trail and handedness!
